# Compression algorithms

### Huffman:

In [94]:
def huffman(data):
    ...

### LZ77 / LZ78

In [95]:
def lz77(data):
    ...

#or

def lz78(data):
    ...

### LZW

In [96]:
class LZW:
    name = 'lzw'
    def compress(self, data: bytes) -> tuple[bytes, list]:
        '''LZW encoding'''
        output = []
        w = b''
        dictionary = {bytes([i]): i for i in range(256)}
        for byte in data:
            byte = bytes([byte])
            wc = w + byte
            if wc in dictionary:
                w = wc
            else:
                output.append(dictionary[w])
                dictionary[wc] = len(dictionary)
                w = byte
        if w:
            output.append(dictionary[w])
        return output, {bytes([i]): i for i in range(256)}

    def decompress(self, code: bytes, coding_dict: dict) -> bytes:
        '''LZW decoding'''
        coding_dict = {v:k for k,v in coding_dict.items()}
        string = coding_dict[code[0]]
        output = bytearray()
        output += string
        for i in range(1, len(code)):
            new = code[i]
            if new not in coding_dict:
                entry = string + string[:1]
            else:
                entry = coding_dict[new]
            output += entry
            coding_dict[len(coding_dict)] = string + entry[:1]
            string = entry
        return bytes(output)

### Deflate

In [97]:
def deflate(data):
    ...

### Other

In [98]:
def other(data):
    ...

In [99]:
def encoding(path:str, compress_algorithm:object):
    with open(path, 'rb') as file:
        image = file.read()
    encoded_data, encoded_dict = compress_algorithm.compress(image)
    file_type = path[::-1].split('.', maxsplit=1)[0][::-1]
    file_path = path[::-1].split('.', maxsplit=1)[1][::-1]+'.'+compress_algorithm.name.lower()
    with open(file_path, 'wb') as file:
        for value in encoded_data:
            file.write(value.to_bytes(3, byteorder='little'))
    return file_path, file_type, compress_algorithm, encoded_dict

def decoding(path:str, f_type:str, compress_algorithm:object, start_dict:dict = None):
    with open(path, 'rb') as file:
        encoded_data = []
        while (byte := file.read(3)):
            encoded_data.append(int.from_bytes(byte, byteorder='little'))
    if start_dict:
        decoded = compress_algorithm.decompress(encoded_data, start_dict)
    else:
        decoded = compress_algorithm.decompress(encoded_data)
    file_path = path[::-1].split('.', maxsplit=1)[1][::-1]+'_decoded.'+f_type
    with open(file_path, 'wb') as file:
        file.write(decoded)